In [ ]:
import cv2
from skimage.feature import local_binary_pattern
import numpy as np
import os

In [ ]:
def COLBP(image:str, num_points=8, radius=1):
    """Compute the Compast local binary patterns (Co-LBP) of an image.

    The Compass Local Binary Pattern (CoLBP) method significantly enhances local binary pattern by integrating directional edge responses with traditional texture analysis techniques.

    Parameters
    ----------
    image : (M, N) array
        2D grayscale image.
    num_points : int
        Number of circularly symmetric neighbor set points (quantization of
        the angular space).
    radius : float
        Radius of circle (spatial resolution of the operator).
    

    Returns
    -------
    output : (D, M, N) array
        Consist of 8 Co-LBP Image.
    """
    image = cv2.imread(image)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY).astype('uint8')

    ea= np.array([[-3,-3,5],[-3,0,5],[-3,-3,5]])
    nea= np.array([[-3,5,5],[-3,0,5],[-3,-3,-3]])
    na= np.array([[5,5,5],[-3,0,-3],[-3,-3,-3]])
    nwa= np.array([[5,5,-3],[5,0,-3],[-3,-3,-3]])
    wa= np.array([[5,-3,-3],[5,0,-3],[5,-3,-3]])
    swa= np.array([[-3,-3,-3],[5,0,-3],[5,5,-3]])
    sa= np.array([[-3,-3,-3],[-3,0,-3],[5,5,5]])
    sea= np.array([[-3,-3,-3],[-3,0,5],[-3,5,5]])
    

    result_na = cv2.filter2D(image, ddepth=-1, kernel=na)
    result_na = local_binary_pattern(result_na, P=num_points, R=radius)

    result_wa = cv2.filter2D(image, ddepth=-1, kernel=wa)
    result_wa = local_binary_pattern(result_wa, P=num_points, R=radius)

    result_sa = cv2.filter2D(image, ddepth=-1, kernel=sa)
    result_sa = local_binary_pattern(result_sa, P=num_points, R=radius)

    result_nea = cv2.filter2D(image, ddepth=-1, kernel=nea)
    result_nea = local_binary_pattern(result_nea, P=num_points, R=radius)

    result_nwa = cv2.filter2D(image, ddepth=-1, kernel=nwa)
    result_nwa = local_binary_pattern(result_nwa, P=num_points, R=radius)

    result_sea = cv2.filter2D(image, ddepth=-1, kernel=sea)
    result_sea = local_binary_pattern(result_sea, P=num_points, R=radius)

    result_swa = cv2.filter2D(image, ddepth=-1, kernel=swa)
    result_swa = local_binary_pattern(result_swa, P=num_points, R=radius)

    result_ea = cv2.filter2D(image, ddepth=-1, kernel=ea)
    result_ea = local_binary_pattern(result_ea, P=num_points, R=radius)
    return result_ea, result_nea, result_na, result_nwa, result_wa, result_swa, result_sa, result_sea


In [ ]:
def extract_colbp_features(paths, num_points=8, radius=1):
    features = []
    for path in paths:
        colbp_features = COLBP(image=path, num_points=num_points, radius=radius)
        feature_vector = []
        
        for result  in colbp_features:
            hist, _ = np.histogram(result.ravel(), 
                                        bins=np.arange(0,257), 
                                        range=(0, 256))
            feature_vector.extend(hist)

        feature_vector = np.array(feature_vector) 
        features.append(feature_vector)    
        
    return np.array(features)

In [ ]:
def extract_colbp_features_from_directory(directory_path, output_file_name):
    categories = { "male" : 0, "female" : 1}

    image_paths = []
    labels = []

    for category, label in categories.items():
        category_path = os.path.join(directory_path, category)
        for img_name in sorted(os.listdir(category_path)):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                img_path = os.path.join(category_path, img_name)    
                image_paths.append(img_path)
                labels.append(label)

    features_array = extract_colbp_features(image_paths=image_paths)
    labels_array = np.array(labels)

    np.savez(f'{output_file_name}.npz', features=features_array, labels=labels_array)


In [ ]:
# scheme 1
extract_colbp_features_from_directory(directory_path="../dataset/scheme-1", output_file_name="odlbp_1")
# scheme 2
extract_colbp_features_from_directory(directory_path="../dataset/scheme-2", output_file_name="odlbp_2")